<a href="https://colab.research.google.com/github/hxnguyen/Tram2Flows/blob/tram2flows_v2/TTPs_to_JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load Rag Result##

In [ ]:
from google.colab import files
import json
uploaded = files.upload()
print(uploaded)

Saving CobaltKitty_op.json to CobaltKitty_op.json
{'CobaltKitty_op.json': b'[{"index":1,"segment":"environment after the company\\u2019s IT department suspected that their network was breached but could not trace the source.","label(s)":["System Network Configuration Discovery - T1016"],"name":"CobaltKitty.docx","assets":[" PowerShell, Cobalt Strike, Word"],"prerequisite":[],"operator":""},{"index":3,"segment":"the threat actor operated a fileless PowerShell-based infrastructure, using customized PowerShell payloads taken from known offensive frameworks such","label(s)":["Obfuscated Files or Information - T1027"],"name":"CobaltKitty.docx","assets":[" Payloads"],"prerequisite":["System Network Configuration Discovery - T1016"],"operator":""},{"index":5,"segment":"out by social engineering. Carefully selected group of employees received spear-phishing emails, containing either links to malicious sites or weaponized Word documents. These documents contained malicious macros that created p

In [ ]:
import pandas as pd
key_ls = list(uploaded.keys())
df = pd.read_json(key_ls[0])

try:
  new_col = {"rag_result": "assets"}
  df.rename(columns=new_col, inplace=True)
except:
  pass


In [ ]:
### Eliminate unsued collums
attack_name = df.loc[0]['name']
df = df.drop(['index', 'name'], axis = 1)


In [ ]:
### Eliminate duplication

from difflib import SequenceMatcher as SM



for i in range(len(df.index)):
  df.loc[i]['assets'] = df.loc[i]['assets'][0].split(',')


for i in range(0, len(df.index)):
  if i >= len(df.index):
    break
  cur = df.loc[i]
  drop_ls = []
  ls = []

  for j in range(2):
    index = i + j + 1
    if index >= len(df.index):
      continue
    fol = df.loc[index]
    if cur['label(s)'] == fol['label(s)']:
      cur_assets = cur['assets']
      fol_assets = fol['assets']
      ls = cur_assets + fol_assets
      # print('-------')
      # print(ls)
      for a1 in cur_assets:
        for a2 in fol_assets:
          asset_rate = SM(None, a1, a2).ratio()

          if asset_rate > 0.7:
            # print(asset_rate)
            ls.remove(a1)

      df.loc[i]['assets'] = ls
      # print(ls, cur_assets, fol_assets)
      # print("-------")

      for lbl in fol['prerequisite']:
        if lbl not in cur['prerequisite']:
          df.loc[i]['prerequisite'].append(lbl)

      df.loc[i]['segment'] += fol['segment']
      drop_ls.append(index)
  # df.loc[i]['assets'] = ls
  df = df.drop(drop_ls)
  df = df.reset_index(drop=True)
df

,segment,label(s),assets,prerequisite,operator
0,environment after the company’s IT department ...,[System Network Configuration Discovery - T1016],"[ PowerShell, Cobalt Strike, Word]",[],
1,the threat actor operated a fileless PowerShel...,[Obfuscated Files or Information - T1027],[ Payloads],[System Network Configuration Discovery - T1016],
2,out by social engineering. Carefully selected ...,[Spearphishing Attachment - T1566.001],"[ Macros, Word documents]",[System Network Configuration Discovery - T101...,I don't know.
3,malicious macros that created persistence on t...,[Scheduled Task - T1053.005],[ Cobalt Strike],[System Network Configuration Discovery - T101...,Yes
4,Uses Javascript to download a Cobalt Strike Be...,[Ingress Tool Transfer - T1105],[ Cobalt Strike Beacon],"[Spearphishing Attachment - T1566.001, Schedul...",I don't know.
5,documents in our Attack Life Cycle section. FI...,[Obfuscated Files or Information - T1027],[ Payload delivery infrastructure.],"[Scheduled Task - T1053.005, Ingress Tool Tran...",Yes
6,and scheduled tasks. This persistence mechanis...,[Scheduled Task - T1053.005],[ PowerShell.],[I don't know the answer],
7,mechanism ensured that the loader scripts woul...,[Registry Run Keys / Startup Folder - T1547.001],[ VBS scripts],"[Obfuscated Files or Information - T1027, Sche...",Yes
8,Values found in Windows’ Registry: the VBS scr...,[Modify Registry - T1112],[ VBS],[Scheduled Task - T1053.005],
9,"the loader’s script, which launches PowerShell...",[Obfuscated Files or Information - T1027],"[ PowerShell, Customized Outlook Credentials ...","[Modify Registry - T1112, System Network Confi...",


In [ ]:
print(df.loc[0]['assets'])

[' PowerShell', ' Cobalt Strike', ' Word']


In [ ]:
### Eliminate Infinite Loops
def eliminate_loop(attackframe):
  for i in range(len(attackframe.index)):
    for pre in attackframe.loc[i]['prerequisite']:
      if pre == attackframe.loc[i]['label(s)']:
        attackframe.loc[i]['label(s)'].remove(pre)

  return attackframe

df = eliminate_loop(df)
df

,segment,label(s),assets,prerequisite,operator
0,environment after the company’s IT department ...,[System Network Configuration Discovery - T1016],"[ PowerShell, Cobalt Strike, Word]",[],
1,the threat actor operated a fileless PowerShel...,[Obfuscated Files or Information - T1027],[ Payloads],[System Network Configuration Discovery - T1016],
2,out by social engineering. Carefully selected ...,[Spearphishing Attachment - T1566.001],"[ Macros, Word documents]",[System Network Configuration Discovery - T101...,I don't know.
3,malicious macros that created persistence on t...,[Scheduled Task - T1053.005],[ Cobalt Strike],[System Network Configuration Discovery - T101...,Yes
4,Uses Javascript to download a Cobalt Strike Be...,[Ingress Tool Transfer - T1105],[ Cobalt Strike Beacon],"[Spearphishing Attachment - T1566.001, Schedul...",I don't know.
5,documents in our Attack Life Cycle section. FI...,[Obfuscated Files or Information - T1027],[ Payload delivery infrastructure.],"[Scheduled Task - T1053.005, Ingress Tool Tran...",Yes
6,and scheduled tasks. This persistence mechanis...,[Scheduled Task - T1053.005],[ PowerShell.],[I don't know the answer],
7,mechanism ensured that the loader scripts woul...,[Registry Run Keys / Startup Folder - T1547.001],[ VBS scripts],"[Obfuscated Files or Information - T1027, Sche...",Yes
8,Values found in Windows’ Registry: the VBS scr...,[Modify Registry - T1112],[ VBS],[Scheduled Task - T1053.005],
9,"the loader’s script, which launches PowerShell...",[Obfuscated Files or Information - T1027],[ PowerShell],[Modify Registry - T1112],


##Structure Parsing##

In [ ]:
def dependency_parsing(attackframe):
  dependency = {}
  for i in range(len(attackframe.index)):
    cur_ttp = attackframe.loc[i]['label(s)'][0] + f' - {i}'
    dependency[cur_ttp] = {}
    dependency[cur_ttp]['assets'] = attackframe.loc[i]['assets'][0].split(", ")
    if dependency[cur_ttp]['assets'][0] == " ":
      dependency[cur_ttp]['assets'] = []
    dependency[cur_ttp]['index'] = i
    dependency[cur_ttp]['prerequisite'] = {}
    dependency[cur_ttp]["operator"] = ""
    if attackframe.loc[i]["operator"].strip() == "Yes":
      dependency[cur_ttp]["operator"] = "AND"
    elif attackframe.loc[i]["operator"].strip()  == "No":
      dependency[cur_ttp]["operator"] = "OR"
    for pre in attackframe.loc[i]['prerequisite']:

      # Identify the prerequisite and its id so as not to confuse with other "same but different" ttps
      for j in range(3):
        index = i - j - 1
        if index < 0:
          continue
        if attackframe.loc[index]['label(s)'][0] == pre:
          dependency[cur_ttp]['prerequisite'][pre] = index


  return dependency

dependency = dependency_parsing(df)
print(dependency)

{'System Network Configuration Discovery - T1016 - 0': {'assets': [' PowerShell', 'Cobalt Strike', 'Word'], 'index': 0, 'prerequisite': {}, 'operator': ''}, 'Obfuscated Files or Information - T1027 - 1': {'assets': [' Payloads'], 'index': 1, 'prerequisite': {'System Network Configuration Discovery - T1016': 0}, 'operator': ''}, 'Spearphishing Attachment - T1566.001 - 2': {'assets': [' Macros', 'Word documents'], 'index': 2, 'prerequisite': {'System Network Configuration Discovery - T1016': 0, 'Obfuscated Files or Information - T1027': 1}, 'operator': ''}, 'Scheduled Task - T1053.005 - 3': {'assets': [' Cobalt Strike'], 'index': 3, 'prerequisite': {'System Network Configuration Discovery - T1016': 0, 'Spearphishing Attachment - T1566.001': 2}, 'operator': 'AND'}, 'Ingress Tool Transfer - T1105 - 4': {'assets': [' Cobalt Strike Beacon'], 'index': 4, 'prerequisite': {'Spearphishing Attachment - T1566.001': 2, 'Scheduled Task - T1053.005': 3}, 'operator': ''}, 'Obfuscated Files or Informat

##Reformatting JSON##


In [ ]:
!pip install datetime
!pip install json
!pip install uuid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 9.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
  Preparing metadata (setup.py) ... done
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6479 sha256=fd63295dc67ca8df7d5e41c07c1c27ec1229e60e61208aadc9fa676237ab176d
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built uuid


In [ ]:
## Add Ids to attack actions and attack assets
import uuid

def add_ids(attackdict):
  for key, value in attackdict.items():
    attackdict[key]["id"] = "attack-action--" + str(uuid.uuid1())
    awid = {}
    for asset in attackdict[key]['assets']:
      awid[asset] = "attack-asset--" + str(uuid.uuid1())
    attackdict[key]['assets'] = awid
    print(awid)
  return attackdict

dependency = add_ids(dependency)
print(dependency)

{' PowerShell': 'attack-asset--276a265a-b039-11ee-9b32-0242ac1c000c', 'Cobalt Strike': 'attack-asset--276a2754-b039-11ee-9b32-0242ac1c000c', 'Word': 'attack-asset--276a2826-b039-11ee-9b32-0242ac1c000c'}
{' Payloads': 'attack-asset--276a603e-b039-11ee-9b32-0242ac1c000c'}
{' Macros': 'attack-asset--276a781c-b039-11ee-9b32-0242ac1c000c', 'Word documents': 'attack-asset--276a7902-b039-11ee-9b32-0242ac1c000c'}
{' Cobalt Strike': 'attack-asset--276aa800-b039-11ee-9b32-0242ac1c000c'}
{' Cobalt Strike Beacon': 'attack-asset--276adf78-b039-11ee-9b32-0242ac1c000c'}
{' Payload delivery infrastructure.': 'attack-asset--276aff8a-b039-11ee-9b32-0242ac1c000c'}
{' PowerShell.': 'attack-asset--276b032c-b039-11ee-9b32-0242ac1c000c'}
{' VBS scripts': 'attack-asset--276b0638-b039-11ee-9b32-0242ac1c000c'}
{' VBS': 'attack-asset--276b08ea-b039-11ee-9b32-0242ac1c000c'}
{' PowerShell': 'attack-asset--276b0bba-b039-11ee-9b32-0242ac1c000c'}
{' Customized Outlook Credentials Dumper.': 'attack-asset--276b0f66-b03

In [ ]:
## Prerequisite to following attacks

# def find_next_attacks(attackdict):
#   start_ref = []
#   for key, value in attackdict.items():
#     attackdict[key]['next_act'] = []
#     ls = attackdict[key]['prerequisite']
#     if len(ls) == 0:
#       start_ref.append(value['id'])
#     for prekey, preval in ls.items():
#       prerequisite_key = prekey + f' - {preval}'
#       attackdict[prerequisite_key]['next_act'].append(value['id'])
#     del attackdict[key]['prerequisite']
#   return attackdict, start_ref

def find_next_attacks(attackdict):
  start_ref = []
  op_ls = {}
  for key, value in attackdict.items():
    attackdict[key]['next_act'] = []
    ls = attackdict[key]['prerequisite']

    if attackdict[key]['operator'] == "AND" or attackdict[key]['operator'] == "OR":
      op_key = attackdict[key]['operator'] + f" - {attackdict[key]['index']}"
      op_ls[op_key] = {}
      op_ls[op_key]['id'] = "attack-operator--" + str(uuid.uuid1())
      op_ls[op_key]['next_act'] = [value['id']]
      op_ls[op_key]['op'] = attackdict[key]['operator']

      op_id = op_ls[op_key]['id']

      for prekey, preval in ls.items():
        prerequisite_key = prekey + f' - {preval}'
        attackdict[prerequisite_key]['next_act'].append(op_id)

    else:
      if len(ls) == 0:
        start_ref.append(value['id'])
      for prekey, preval in ls.items():
        prerequisite_key = prekey + f' - {preval}'
        attackdict[prerequisite_key]['next_act'].append(value['id'])

    # del attackdict[key]['prerequisite']
    # del attackdict[key]['operator']

  # print(op_ls)
  attackdict.update(op_ls)

  return attackdict, start_ref

dependency, start_ref = find_next_attacks(dependency)

print(dependency)

{'System Network Configuration Discovery - T1016 - 0': {'assets': {' PowerShell': 'attack-asset--276a265a-b039-11ee-9b32-0242ac1c000c', 'Cobalt Strike': 'attack-asset--276a2754-b039-11ee-9b32-0242ac1c000c', 'Word': 'attack-asset--276a2826-b039-11ee-9b32-0242ac1c000c'}, 'index': 0, 'prerequisite': {}, 'operator': '', 'id': 'attack-action--276a23da-b039-11ee-9b32-0242ac1c000c', 'next_act': ['attack-action--276a5e7c-b039-11ee-9b32-0242ac1c000c', 'attack-action--276a768c-b039-11ee-9b32-0242ac1c000c', 'attack-operator--2bb1432e-b039-11ee-9b32-0242ac1c000c']}, 'Obfuscated Files or Information - T1027 - 1': {'assets': {' Payloads': 'attack-asset--276a603e-b039-11ee-9b32-0242ac1c000c'}, 'index': 1, 'prerequisite': {'System Network Configuration Discovery - T1016': 0}, 'operator': '', 'id': 'attack-action--276a5e7c-b039-11ee-9b32-0242ac1c000c', 'next_act': ['attack-action--276a768c-b039-11ee-9b32-0242ac1c000c']}, 'Spearphishing Attachment - T1566.001 - 2': {'assets': {' Macros': 'attack-asset--

##TEMPLATEs##

##Converting

In [ ]:
## converting to JSON-like attackflow
from datetime import datetime

def convertDictKeys2Nodes(attackdict, extension):
  Nodes = []
  cd = datetime.now()
  current_date = cd.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
  for key,value in attackdict.items():
    name = key.split("-")[0].strip()
    if name == "AND" or name == "OR":
      operator = {
        "type": "attack-operator",
        "id": value['id'],
        "spec_version": "2.1",
        "created": current_date,
        "modified": current_date,
        "extensions": extension,
        "operator": name,
        "effect_refs": value['next_act']
      }
      if len(operator['effect_refs']) == 0:
        del operator['effect_refs']
      Nodes.append(operator)
    else:
      tech_id = key.split("-")[1].strip()

      action = {
        "type": "attack-action",
        "id": value['id'],
        "spec_version": "2.1",
        "created": current_date,
        "modified": current_date,
        "extensions": extension,
        "name": name,
        "tactic_id": tech_id,
        "confidence": 100,
        "effect_refs": value['next_act'],
        "asset_refs": list(value['assets'].values())
      }
      if len(action['effect_refs']) == 0:
        del action['effect_refs']
      if len(action['asset_refs']) == 0:
        del action['asset_refs']

      Nodes.append(action)
      for akey, aval in value['assets'].items():
        asset = {
          "type": "attack-asset",
          "id": aval,
          "spec_version": "2.1",
          "created": current_date,
          "modified": current_date,
          "extensions": extension,
          "name": akey
        }
        Nodes.append(asset)
  return Nodes


def converting2attackflow(attackdict, start_ref):
  cd = datetime.now()
  current_date = cd.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
  bundle_id = "bundle--" + str(uuid.uuid1())
  extension_def_id = "extension-definition--fb9c968a-745b-4ade-9b25-c324172197f4"
  af_id = "attack-flow--" + str(uuid.uuid1())

  #Outer node
  attack_flow_bundle = {
    "type": "bundle",
    "id": bundle_id,
    "spec_version": "2.1",
    "created": current_date,
    "modified": current_date,
    "objects": []
  }

  ## extension definition
  extension_def = {
      "type": "extension-definition",
      "id": extension_def_id,
      "spec_version": "2.1",
      "created": "2022-08-02T19:34:35.143Z",
      "modified": "2022-08-02T19:34:35.143Z",
      "name": "Attack Flow",
      "description": "Extends STIX 2.1 with features to create Attack Flows.",
      "created_by_ref": "identity--fb9c968a-745b-4ade-9b25-c324172197f4",
      "schema": "https://center-for-threat-informed-defense.github.io/attack-flow/stix/attack-flow-schema-2.0.0.json",
      "version": "2.0.0",
      "extension_types": [
        "new-sdo"
      ],
      "external_references": [
        {
          "source_name": "Documentation",
          "description": "Documentation for Attack Flow",
          "url": "https://center-for-threat-informed-defense.github.io/attack-flow"
        },
        {
          "source_name": "GitHub",
          "description": "Source code repository for Attack Flow",
          "url": "https://github.com/center-for-threat-informed-defense/attack-flow"
        }
      ]
  }
  attack_flow_bundle['objects'].append(extension_def)

  extension = {
        f"{extension_def_id}": {
          "extension_type": "new-sdo"
        }
      }

  ##identity
  identity = {
      "type": "identity",
      "id": "identity--fb9c968a-745b-4ade-9b25-c324172197f4",
      "spec_version": "2.1",
      "created": "2022-08-02T19:34:35.143Z",
      "modified": "2022-08-02T19:34:35.143Z",
      "create_by_ref": "identity--fb9c968a-745b-4ade-9b25-c324172197f4",
      "name": "MITRE Engenuity Center for Threat-Informed Defense",
      "identity_class": "organization"
  }
  attack_flow_bundle['objects'].append(identity)

  ##identity_me
  my_id = "identity--" + str(uuid.uuid1())
  id_me = {
      "type": "identity",
      "id": my_id,
      "spec_version": "2.1",
      "created": "2023-06-27T01:41:48.030Z",
      "modified": "2023-06-27T01:41:48.030Z",
      "name": "BoCauHamThuocBac"
  }

  ## attack flow element
  attack_flow = {
      "type": "attack-flow",
      "id": af_id,
      "spec_version": "2.1",
      "created": current_date,
      "modified": current_date,
      "extensions": extension,
      "created_by_ref": my_id,
      "start_refs": start_ref,
      "name": attack_name,
      "scope": "incident"
  }

  attack_flow_bundle['objects'].append(attack_flow)
  attack_flow_bundle['objects'].append(id_me)
  ## other elements
  actions_and_assets = convertDictKeys2Nodes(attackdict, extension)
  print(actions_and_assets)
  attack_flow_bundle["objects"] = attack_flow_bundle["objects"] + actions_and_assets

  return attack_flow_bundle

attack_flow = converting2attackflow(dependency, start_ref)


print(attack_flow)


[{'type': 'attack-action', 'id': 'attack-action--276a23da-b039-11ee-9b32-0242ac1c000c', 'spec_version': '2.1', 'created': '2024-01-11T04:32:09.185418Z', 'modified': '2024-01-11T04:32:09.185418Z', 'extensions': {'extension-definition--fb9c968a-745b-4ade-9b25-c324172197f4': {'extension_type': 'new-sdo'}}, 'name': 'System Network Configuration Discovery', 'tactic_id': 'T1016', 'confidence': 100, 'effect_refs': ['attack-action--276a5e7c-b039-11ee-9b32-0242ac1c000c', 'attack-action--276a768c-b039-11ee-9b32-0242ac1c000c', 'attack-operator--2bb1432e-b039-11ee-9b32-0242ac1c000c'], 'asset_refs': ['attack-asset--276a265a-b039-11ee-9b32-0242ac1c000c', 'attack-asset--276a2754-b039-11ee-9b32-0242ac1c000c', 'attack-asset--276a2826-b039-11ee-9b32-0242ac1c000c']}, {'type': 'attack-asset', 'id': 'attack-asset--276a265a-b039-11ee-9b32-0242ac1c000c', 'spec_version': '2.1', 'created': '2024-01-11T04:32:09.185418Z', 'modified': '2024-01-11T04:32:09.185418Z', 'extensions': {'extension-definition--fb9c968a-7

##Download file##

In [ ]:
from google.colab import files
import json

with open('ANU_op.json', 'w') as file:
    json.dump(attack_flow, file)

files.download('ANU_op.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>